In [1]:
from torchvision import datasets, transforms
import torch
from net import AlexNetPlusLatent
import numpy as np

In [2]:
bits = 48
pretrained = 91.33

In [3]:
net = AlexNetPlusLatent(bits)
net.load_state_dict(torch.load('./model/91.33'))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Use device: ' + str(device))
net.to(device)
net.eval()

Use device: cuda:0


AlexNetPlusLatent(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (remain): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p

In [4]:
def load_data():
    transform_test = transforms.Compose(
        [
         transforms.Resize(227),
         transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ]
    )
    testset = datasets.CIFAR10(root='./data', train=False, download=True,
                               transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset)
    return testloader

In [6]:
def int32_aggregate_output(_loader):
    a = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(_loader):
            _int = int32_single_output(inputs)
            b = (batch_idx, _int, int(targets))
            if batch_idx%500==0:
                print('#', end='')
            a.append(b)
        return a

In [7]:
def single_output(data):
    inputs = data.to(device)
    outputs, _ = net(inputs)
    return outputs.data

In [8]:
def int32_single_output(_data):
    _tensor = (single_output(_data)[0,:]>0.5).int()
    _str = ''
    for i in range(0, _tensor.shape[0]):
        _str += str(int(_tensor[i]))
    _int = int(_str, 2)
    return _int

In [9]:
_loader = load_data()
a = int32_aggregate_output(_loader)

Files already downloaded and verified
####################

In [37]:
a[6569][1]

31598448389273

In [50]:
def hammingDistance(x, y):
    ans = 0
    for i in range(31,-1,-1):
        b1= x>>i&1
        b2 = y>>i&1
        ans+= not(b1==b2)
    return ans

In [51]:
def test(pic_id, hash_list):
    _a = []
    _pic_hash = hash_list[pic_id][1]
    for i in range(0, len(hash_list)):
        _h_distance = hammingDistance(_pic_hash, hash_list[i][1])
        b = (hash_list[i][0], _h_distance, hash_list[i][2])
        _a.append(b)
    return _a

In [69]:
c = test(4567 , a)

In [70]:
# take second element for sort
def takeSecond(elem):
    return elem[1]

# sort list with key
c.sort(key=takeSecond)

# print list
c

[(22, 0, 4),
 (36, 0, 4),
 (94, 0, 4),
 (110, 0, 4),
 (278, 0, 4),
 (479, 0, 4),
 (509, 0, 4),
 (555, 0, 4),
 (627, 0, 4),
 (807, 0, 4),
 (849, 0, 4),
 (1063, 0, 4),
 (1293, 0, 4),
 (1303, 0, 4),
 (1336, 0, 4),
 (1345, 0, 4),
 (1554, 0, 4),
 (1609, 0, 4),
 (1645, 0, 4),
 (1777, 0, 4),
 (1798, 0, 4),
 (1910, 0, 4),
 (1952, 0, 4),
 (2025, 0, 4),
 (2180, 0, 4),
 (2182, 0, 4),
 (2218, 0, 4),
 (2260, 0, 4),
 (2353, 0, 4),
 (2406, 0, 4),
 (2501, 0, 4),
 (2577, 0, 4),
 (2664, 0, 4),
 (2699, 0, 4),
 (2763, 0, 4),
 (2805, 0, 4),
 (2971, 0, 4),
 (2982, 0, 4),
 (3127, 0, 4),
 (3252, 0, 4),
 (3296, 0, 4),
 (3360, 0, 4),
 (3361, 0, 4),
 (3419, 0, 4),
 (3478, 0, 4),
 (3512, 0, 4),
 (3621, 0, 4),
 (3671, 0, 4),
 (3692, 0, 4),
 (3759, 0, 4),
 (3790, 0, 4),
 (3914, 0, 4),
 (3990, 0, 4),
 (4029, 0, 4),
 (4053, 0, 4),
 (4138, 0, 4),
 (4159, 0, 4),
 (4184, 0, 4),
 (4200, 0, 4),
 (4205, 0, 4),
 (4262, 0, 4),
 (4350, 0, 4),
 (4377, 0, 4),
 (4399, 0, 4),
 (4443, 0, 4),
 (4567, 0, 4),
 (4573, 0, 4),
 (4576, 0